# Semi-Supervised Learning Tutorial for Molecular Property Prediction
#### 제작 : 허종국 (hjkso1406@korea.ac.kr)

### Introduction
본 튜토리얼에서는 준지도학습 알고리즘 중 [Fixmatch](https://arxiv.org/ftp/arxiv/papers/2001/2001.07685.pdf)를 활용해 물성을 예측해보도록 하겠습니다. 물성 예측 벤치마크인 [MoleculeNet Benchmark](https://pubs.rsc.org/en/content/articlehtml/2018/sc/c7sc02664a)는 뇌혈관장벽 투과성, 용해도, 전기음성도 등 양자역학, 물리화학, 생물물리학, 생리학에 아우르는 다양한 물성에 대한 데이터셋을 제공합니다.
![Moleculenet](./images/moleculenet.png)

MoleculeNet Benchmark의 타겟은 이진 분류 혹은 연속형 회귀 분석으로 나뉩니다. Fixmatch는 분류 문제를 타겟으로 나온 알고리즘이기 때문에 본 튜토리얼에서는 이진 분류 데이터셋 중 하나인 __BACE__ 데이터셋에 대해 Fixmatch를 구현하고자 합니다. 또한 Unlabeled Dataset으로는 Moleculenet Benchmark 의 multi-target regression 데이터 중 하나인 __QM9__ 데이터를 사용하였습니다. BACE 데이터는 총 1513개로 8:1:1 비율로 학습/검증/테스트 데이터를 분할하였으며, QM9 데이터는 130829개 중 일부를 Unlabeled Data로 사용하였습니다.

### Requiremnets
#### rdkit

__python 3.7이하__
rdkit 패키지의 설치 명령어는 Python version에 따라 다릅니다.
```
pip install rdkit 
```
__python 3.8__
```
conda install -c conda-forge rdkit
```

### Download Data
[링크](https://drive.google.com/file/d/1aDtN6Qqddwwn2x612kWz9g0xQcuAtzDE/view)를 통해 데이터를 다운받으시길 바랍니다. 혹은 `./data` 라는 폴더를 생성한 후 직접 [MoleculeNet](https://moleculenet.org/)에서 다운 받으실 수 있습니다.

### Packages & Hyper-parameters
* MU - 한 Iteration에서 Labeled Data 개수 대비 Unlabeled Data 개수의 비율을 나타냅니다.
* WEIGHT - Unlabeled Loss를 나타냅니다.

In [1]:
import os
import numpy as np
from tqdm import tqdm
from collections import namedtuple
from sklearn.metrics import roc_auc_score as auc

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit import RDLogger

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_geometric.loader import DataLoader
from torch_geometric.data.batch import Batch
from torch.utils.tensorboard import SummaryWriter

from auglichem.molecule import Compose, RandomAtomMask, RandomBondDelete, MotifRemoval
from auglichem.molecule.data import MoleculeDatasetWrapper, MoleculeDataset

import itertools

from model import *
from utils import *
from dataset import DualMoleculeDataset
from loader import DualDataLoader
import warnings

warnings.filterwarnings('ignore')
RDLogger.DisableLog('rdApp.*')

MU = 7
BATCH_SIZE = 64
THRESHOLD = 0.7
WEIGHT = 3.0
EPOCHS = 100

### Model (GINE)
GINE는 Graph Neural Network(GNN) 중 하나로써, [Strategies for Pretraining Graph Neural Netwroks](https://arxiv.org/pdf/1905.12265.pdf) 라는 논문에서 처음 제안되었습니다. 해당 모델은 [How Powerful Are Graph Neural Networks](https://arxiv.org/pdf/1810.00826.pdf)에서 제안한 Graph Isomorphism Network(GIN)을 개선한 모델입니다. 기존의 Graph Convolution 혹은 Message Passing이 __Node Attribute__ 만 고려하였다면, GINE는 __Edge Attribute__ 까지 고려하여 Node Representation을 업데이트한다는 것이 특징입니다.

$$ \mathbf{x}_{i}^{\prime}=h_{\boldsymbol{\Theta}}\left((1+\epsilon) \cdot \mathbf{x}_{i}+\sum_{j \in \mathcal{N}(i)} \operatorname{ReLU}\left(\mathbf{x}_{j}+\mathbf{e}_{j, i}\right)\right) $$

* 출처 - https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GINEConv

이러한 GINE의 특성은 다양한 종류와 그에 따라 상이한 성질을 가지는 원자간 결합(Edge)를 반영하기 적합하기 때문에 본 튜토리얼에서 모델로 차용하였습니다. GINE 모델 구현이 궁금하신 분은 model.py를 참조해주시면 되겠습니다.

In [2]:
model = GINE(drop_ratio=0.3)
device = torch.device('cuda:0')
# device = torch.device('cpu')

### Define Weak & Strong Augmentation

분자를 그래프로 표현하였을 경우 데이터 증강 기법은 크게 아래 3가지로 나눌 수 있습니다.

![Augmentation](./images/Augmentation.png)

Fixmatch를 적용하기 위해서는 이미지와 같이 그래프에서도 Weak Augmentation과 Strong Augmentation을 적용해야합니다. 본 튜토리얼에서는 __결합 정보 보존 유무__ 에 따라 Weak와 Strong Augmentation을 정의하였습니다. 따라서 본 튜토리얼에서는 __atom masking__ 을 Weak Augmentation으로, __atom & bond masking__ 을 Strong Augmentation으로 정의하였습니다. 이외에도 __AugliChem__ 패키지에서 제공하는 Motif(Sub-graph) Removal 를 Strong Augmentation으로 정의할 수 도 있습니다.

In [ ]:
w_aug = Compose([RandomAtomMask([0.1, 0.3])])
s_aug1 = Compose([RandomAtomMask([0.1, 0.3]),
                  RandomBondDelete([0.1, 0.3])])

### Define Custom Dataset & DataLoader
그래프 데이터는 배치를 구성하는데 있어서 이미지와 전혀 다른 프로세스를 가집니다. 이미지의 경우, __(C, H, W)__ 차원의 __N__ 개의 텐서를 스태킹하여 (N, C, H, W)의 4차원 텐서로 변환합니다. 하지만 그래프 데이터의 경우는 이러한 방식으로 텐서를 스태킹할 수 없습니다. 그래프 데이터에서 배치 구성은 N개의 그래프에 대한 __Node Attribute, Edge Attribute, Adjacency Matrix__ 를 __모두 하나의 그래프__ 로 만듭니다. 자세한 사항은 [링크](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html)를 참조해주세요!

그래프 데이터에서 Fixmatch를 적용하기 위해 Weak Augmentation과 Strong Augmentation된 배치가 병렬적으로 생성될 수 있어야합니다. __AugliChem__ 패키지의 __MoleculeDataset__ 을 변형하여 커스텀 데이터셋/데이터로더인 __DualMoleculeDataset__ 과 __DualDataLoader__ 를 구현하였습니다. 자세한 사항은 dataset.py와 dataloader.py를 참조해주세요!!

* Caution : 기존의 MoleculeNet Benchmark의 대다수 데이터는 __Random Split__ 이 아닌 __Scaffold Split__ 을 사용하는 것이 원칙입니다. 학습/검증/테스트 데이터가 서로 다른 __Scaffold__ 를 가지도록하여 __Out-of-Distribution(OOD)__ 상황에서도 강건하게 예측해야하기 떄문입니다. 하지만 본 튜토리얼에서는 Class Distribution의 Mismatch나 OOD를 해결하는 것이 목적이 아니기 때문에, 원활한 학습을 위해 Random Split을 사용합니다.

#### Scaffold Split이란??
스캐폴드(Scaffold)란 분자를 이루고 있는 기본적인 골격 뼈대를 의미합니다. 각 분자별로 가지고 있는 Scaffold는 같을 수도 있고 다를 수도 있습니다. 스캐폴드의 예시는 아래 그림과 같습니다.
![Scaffold](./images/scaffold.png)

In [ ]:
labeled_dataset = MoleculeDatasetWrapper(dataset="BACE",
                                         data_path='./data',
                                         transform=w_aug,
                                         split="random",
                                         batch_size=BATCH_SIZE)
train_loader_labeled, val_loader, test_loader = labeled_dataset.get_data_loaders()

unlabeled_dataset = DualMoleculeDataset(dataset="QM9",
                                    data_path='./data',
                                    s_transform=s_aug1,
                                    w_transform=w_aug,
                                    test_mode=False,
                                    _training_set=True,
                                    _train_warn=False,)
train_loader_unlabeled = DualDataLoader(unlabeled_dataset, batch_size = MU * BATCH_SIZE, drop_last=False, shuffle=False)

### Define Trainer(FixMatch)
![FixMatch](./images/fixmatch.png)
Fixmatch는 두 가지 학습을 동시에 진행합니다.

1.
    Labeled Data에 대해 Weak Augmentation을 수행한 후 지도학습 수행


2.
    Unlabeled Data에 대해 Strong Augmentation과 Weak Augmentation을 적용후 예측
    Weak Augmentation의 예측 값에 대해 Threshold 이상의 Confidency를 가질 경우, Pseudo-label로 지정
    Strong Augmentation의 예측 값에 대해 Pseudo-label을 따라가도록 학습

In [4]:
def _train_step_semi(model: nn.Module,
                     batch: Batch,
                     meters: AverageMeterSet,
                     device:torch.device):
    
    l_batch, (u_w, u_s) = batch
    x_w, labels = l_batch, l_batch.y.reshape(-1)
    
    labels = labels.to(device)
    total_x_batch = list(itertools.chain.from_iterable([b.to_data_list() for b in [l_batch, u_w, u_s]]))
    total_x_batch = Batch.from_data_list(total_x_batch).to(device)
    
    _, total_logits = model(total_x_batch)
    logits_x = total_logits[:len(l_batch)]
    logits_u_w, logits_u_s = total_logits[len(l_batch):].chunk(2)
    
    labeled_loss = F.cross_entropy(logits_x, labels, reduction='mean')
    
    with torch.no_grad():
        pseudo_labels = torch.softmax(logits_u_w, dim=1)
        max_probs, targets_u = torch.max(pseudo_labels, dim=1)
        mask = max_probs.ge(THRESHOLD).float()
    
    unlabeled_loss = (F.cross_entropy(logits_u_s, targets_u, reduction="none") * mask).mean()
    
    loss = labeled_loss.mean() + WEIGHT * unlabeled_loss
    
    meters.update("total_loss", loss.item(), 1)
    meters.update("labeled_loss", labeled_loss.mean().item(), logits_x.size()[0])
    meters.update("unlabeled_loss", unlabeled_loss.item(), logits_u_s.size()[0])
    
    return loss

In [5]:
def train_epoch_semi(model: nn.Module,
                     ema_model: EMA,
                     train_loader_labeled: DataLoader,
                     train_loader_unlabeled: DataLoader,
                     optimizer: torch.optim.Adam,
                     scaler: GradScaler,
                     device: torch.device):
    
    meters = AverageMeterSet()
    
    model.train()
    with tqdm(**get_tqdm_config(total=len(train_loader_labeled), leave=True, color='cyan')) as pbar:
        for idx, batch in enumerate(zip(train_loader_labeled, train_loader_unlabeled)):
            with autocast():
                loss = _train_step_semi(model, batch, meters, device)
            
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            ema_model(model)
            pbar.set_description(f"[Train(Batch)-- Idx: {idx + 1}| Train(Total) Loss : {meters['total_loss'].avg:.4f} | Train(Labeled) Loss : {meters['labeled_loss'].avg:.4f} | Train(Unlabeled) Loss : {meters['unlabeled_loss'].avg}")
            pbar.update(1)
        pbar.set_description(f"[Train(Epoch)-- Idx: {idx + 1}| Train(Total) Loss : {meters['total_loss'].avg:.4f} | Train(Labeled) Loss : {meters['labeled_loss'].avg:.4f} | Train(Unlabeled) Loss : {meters['unlabeled_loss'].avg}")
    
    return meters['total_loss'].avg, meters['labeled_loss'].avg, meters['unlabeled_loss'].avg  

### Evaluation Metrics
평가지표로는 ROC-AUC를 사용하였습니다.

In [6]:
evaluation_metrics = namedtuple(
    "evaluation_metrics",
    ["loss", "roc_auc"],)

@torch.no_grad()
def evaluate(model:nn.Module,
             loader:DataLoader,
             device:torch.device,
             desc:str):

    preds_logit = []
    targets_binary = []
    
    meters = AverageMeterSet()
    
    model.eval()
    with tqdm(**get_tqdm_config(total=len(loader), leave=True, color='magenta')) as pbar:
        for idx, batch in enumerate(loader):
            batch = batch.to(device)
            _, pred = model(batch)
            loss = F.cross_entropy(pred, batch.y.reshape(-1), reduction='mean')

            preds_logit.extend(F.softmax(pred, dim=-1).detach().cpu().numpy())
            targets_binary.extend(batch.y.reshape(-1).detach().cpu().numpy())
            
            meters.update("loss", loss.item(), batch.y.shape[0])

            pbar.set_description(f"[Val(Batch)-- Idx: {idx + 1}| {desc} Loss : {meters['loss'].avg:.4f}]")
            pbar.update(1)
            
        rocauc = auc(targets_binary, np.array(preds_logit)[:, 1])
        meters.update("roc_auc", rocauc, len(targets_binary))
        pbar.set_description(f"[Val(Epoch)-- Idx: {idx + 1}| {desc} Loss : {meters['loss'].avg:.4f} | {desc} ROC-AUC : {meters['roc_auc'].avg:.4f}]")
       
    metrics = evaluation_metrics(loss=meters["loss"].avg, roc_auc=meters["roc_auc"].avg)
    return metrics

### Result of FixMatch
Fixmatch로 GINE를 학습하고, Best Validation 모델에 대해 Test Data ROC-AUC를 구하였습니다.

In [7]:
def run_semi(model: nn.Module,
            train_loader_labeled: DataLoader,
            train_loader_unlabeled: DataLoader,
            val_loader: DataLoader,
            test_loader: DataLoader,
            writer: SummaryWriter,
            device: torch.device,
            ema_decay: float,
            epochs: int=100,
            exp_id: int=1):
      
      model.to(device)
      ema_model = EMA(model, ema_decay)
      
      optim_params = get_wd_param_list(model)      
      optimizer = torch.optim.Adam(optim_params, lr=5e-4, weight_decay=1e-5)
      scaler = GradScaler()
      scheduler = CosineAnnealingLR(optimizer, T_max=90, eta_min=0, last_epoch=-1)
      if not os.path.exists(f'./ckpt/EXP_{exp_id}'):
            os.makedirs(f'./ckpt/EXP_{exp_id}')
      best_val_cls = 0.
      for ep in range(epochs):
            print(f"[Epoch : {ep:03d}]")
            train_total_loss, train_labeled_loss, train_unlabeled_loss = train_epoch_semi(model,
                                                                                          ema_model,
                                                                                          train_loader_labeled,
                                                                                          train_loader_unlabeled,
                                                                                          optimizer,
                                                                                          scaler,
                                                                                          device)
            ema_model.assign(model)
            val_metrics = evaluate(model, val_loader, device, "Validation")
            writer.add_scalar("Loss/train_total", train_total_loss, ep)
            writer.add_scalar("Loss/train_labeled", train_labeled_loss, ep)
            writer.add_scalar("Loss/train_unlabeled", train_unlabeled_loss, ep)
            writer.add_scalar("Loss/val_total", val_metrics.loss, ep)
            writer.add_scalar("Classification_Metrics/auc", val_metrics.roc_auc, ep)
            ema_model.resume(model)
            
            if val_metrics.roc_auc > best_val_cls:
                  best_val_cls = val_metrics.roc_auc
                  torch.save(model.state_dict(), './ckpt/EXP_{exp_id}/best_model.ckpt')
            
            if ep > 10:
                  scheduler.step()
            
      best_ckpt = torch.load(f'./ckpt/EXP_{exp_id}/best_model.ckpt')
      model.load_state_dict(best_ckpt)
      model.eval()
      test_metrics = evaluate(model, test_loader, device, "Test")
      return test_metrics

In [8]:
writer = SummaryWriter('./runs/EXP_1')
test_metrics = run_semi(model, train_loader_labeled, train_loader_unlabeled, val_loader, test_loader, writer, device, ema_decay=0.999, epochs=EPOCHS)

[Epoch : 000]
[Train(Epoch)-- Idx: 19| Train(Total) Loss : 0.7198 | Train(Labeled) Loss : 0.7007 | Train(Unlabeled) Loss : 0.006398660185942917: 100%|██████████| [00:15<00:00,  1.26it/s]
[Val(Epoch)-- Idx: 3| Validation Loss : 0.6854 | Validation ROC-AUC : 0.6941]: 100%|██████████| [00:00<00:00, 20.27it/s]
[Epoch : 001]
[Train(Epoch)-- Idx: 19| Train(Total) Loss : 0.6895 | Train(Labeled) Loss : 0.6883 | Train(Unlabeled) Loss : 0.00040891835171925396: 100%|██████████| [00:14<00:00,  1.28it/s]
[Val(Epoch)-- Idx: 3| Validation Loss : 0.6839 | Validation ROC-AUC : 0.5481]: 100%|██████████| [00:00<00:00, 21.28it/s]
[Epoch : 002]
[Train(Epoch)-- Idx: 19| Train(Total) Loss : 0.6822 | Train(Labeled) Loss : 0.6799 | Train(Unlabeled) Loss : 0.0007862005333759283: 100%|██████████| [00:14<00:00,  1.32it/s] 
[Val(Epoch)-- Idx: 3| Validation Loss : 0.6849 | Validation ROC-AUC : 0.6193]: 100%|██████████| [00:00<00:00, 20.54it/s]
[Epoch : 003]
[Train(Epoch)-- Idx: 19| Train(Total) Loss : 0.6822 | Trai

테스트 데이터 성능 결과 ROC-AUC가 0.8720을 기록하였습니다. Fixmatch의 효과를 입증하기 위해서는 Unlabeled Data를 사용하지 않은 순수 지도학습의 결과가 필요합니다.

In [9]:
print(f"Test Performance : ROC-AUC {test_metrics.roc_auc:.4f}")

Test Performance : ROC-AUC 0.8720


### Define Trainer(Supervised Learning)

In [32]:
def _train_step(model: nn.Module,
                batch: Batch,
                meters: AverageMeterSet,
                device:torch.device):
    
    batch = batch.to(device)
    
    _, pred = model(batch)
    
    labeled_loss = F.cross_entropy(pred, batch.y.reshape(-1), reduction='mean')
    
    meters.update("labeled_loss", labeled_loss.mean().item(), pred.shape[0])
    
    return labeled_loss

def train_epoch(model: nn.Module,
                train_loader_labeled: DataLoader,
                optimizer: torch.optim.Adam,
                scaler: GradScaler,
                device: torch.device):
    
    meters = AverageMeterSet()
    
    model.train()
    with tqdm(**get_tqdm_config(total=len(train_loader_labeled), leave=True, color='cyan')) as pbar:
        for idx, batch in enumerate(train_loader_labeled):
            with autocast():
                loss = _train_step(model, batch, meters, device)
            
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            pbar.set_description(f"[Train(Batch)-- Idx: {idx + 1}| Train(Labeled) Loss : {meters['labeled_loss'].avg:.4f}")
            pbar.update(1)
        pbar.set_description(f"[Train(Epoch)-- Idx: {idx + 1}| Train(Labeled) Loss : {meters['labeled_loss'].avg:.4f}")
    
    return meters['labeled_loss'].avg

def run(model: nn.Module,
        train_loader_labeled: DataLoader,
        val_loader: DataLoader,
        test_loader: DataLoader,
        writer: SummaryWriter,
        device: torch.device,
        epochs: int=100,
        exp_id: int=1):
    
    model.to(device)
      
    optim_params = get_wd_param_list(model)      
    optimizer = torch.optim.Adam(optim_params, lr=5e-4, weight_decay=1e-5)
    scaler = GradScaler()
    scheduler = CosineAnnealingLR(optimizer, T_max=90, eta_min=0, last_epoch=-1)
    if not os.path.exists(f'./ckpt/EXP_{exp_id}'):
        os.makedirs(f'./ckpt/EXP_{exp_id}')
        
    best_val_cls = 0.
    for ep in range(epochs):
        print(f"[Epoch : {ep:03d}]")
        train_labeled_loss = train_epoch(model,
                                         train_loader_labeled,
                                         optimizer,
                                         scaler,
                                         device)
        val_metrics = evaluate(model, val_loader, device, "Validation")
        writer.add_scalar("Loss/train_labeled", train_labeled_loss, ep)
        writer.add_scalar("Loss/val_total", val_metrics.loss, ep)
        writer.add_scalar("Classification_Metrics/auc", val_metrics.roc_auc, ep)

        if val_metrics.roc_auc > best_val_cls:
            best_val_cls = val_metrics.roc_auc
            torch.save(model.state_dict(), f'./ckpt/EXP_{exp_id}/best_model.ckpt')
            
            if ep > 10:
                scheduler.step()
            
    best_ckpt = torch.load(f'./ckpt/EXP_{exp_id}/best_model.ckpt')
    model.load_state_dict(best_ckpt)
    model.eval()
    test_metrics = evaluate(model, test_loader, device, "Test")
    return test_metrics

### Result of Supervised Learning

In [33]:
writer = SummaryWriter('./runs/EXP_0')
model_sup = GINE(drop_ratio=0.3)
test_metrics = run(model_sup, train_loader_labeled, val_loader, test_loader, writer, device, epochs=EPOCHS, exp_id=0)

[Epoch : 000]
[Train(Epoch)-- Idx: 19| Train(Labeled) Loss : 0.6837: 100%|██████████| [00:01<00:00, 11.66it/s]
[Val(Epoch)-- Idx: 3| Validation Loss : 0.6965 | Validation ROC-AUC : 0.5259]: 100%|██████████| [00:00<00:00, 21.28it/s]
[Epoch : 001]
[Train(Epoch)-- Idx: 19| Train(Labeled) Loss : 0.6301: 100%|██████████| [00:01<00:00, 11.93it/s]
[Val(Epoch)-- Idx: 3| Validation Loss : 0.6486 | Validation ROC-AUC : 0.6889]: 100%|██████████| [00:00<00:00, 18.99it/s]
[Epoch : 002]
[Train(Epoch)-- Idx: 19| Train(Labeled) Loss : 0.5969: 100%|██████████| [00:01<00:00, 12.64it/s]
[Val(Epoch)-- Idx: 3| Validation Loss : 0.5385 | Validation ROC-AUC : 0.8682]: 100%|██████████| [00:00<00:00, 20.69it/s]
[Epoch : 003]
[Train(Epoch)-- Idx: 19| Train(Labeled) Loss : 0.5928: 100%|██████████| [00:01<00:00, 12.11it/s]
[Val(Epoch)-- Idx: 3| Validation Loss : 0.6291 | Validation ROC-AUC : 0.7753]: 100%|██████████| [00:00<00:00, 17.75it/s]
[Epoch : 004]
[Train(Epoch)-- Idx: 19| Train(Labeled) Loss : 0.5780: 100

지도 학습 기준 ROC-AUC가 0.8663을 기록하였습니다. 0.006의 상승폭을 보였으나, 단일 실험 결과이기 때문에 유의미한 결과라고 단정짓기는 어려울 것 같습니다.

In [34]:
print(f"Test Performance : ROC-AUC {test_metrics.roc_auc:.4f}")

Test Performance : ROC-AUC 0.8663


### Conclusion
본 튜토리얼에서는 Fixmatch를 활용하여 BACE 데이터셋에서 Unlabeled Data(QM9)을 사용하였을 경우 효과를 나타내는지에 대해 알아보았습니다. 결론은 아래와 같습니다.

1. 소폭의 상승이 있었으나, 반복 실험을 통해 효과를 입증해야할 필요가 있습니다.
2. 성능 상승 폭이 크지 않은 이유 중 하나로 BACE와 QM9 데이터셋에 들어있는 분자의 분포가 상이하기 때문이라고 추측합니다. __베타-세크리테이즈__ 라는 효소에 대해 억제성을 가지고 있는지에 대해 레이블이 되어있으며, __생물물리학__ 에 기반한 데이터셋인 반면, QM9 데이터의 경우 __양자역학__ 에 기반하여 데이터가 구축되어 있습니다. 따라서 차후 개선점으로는 QM9이 아니라 생물물리학 기반의 다른 데이터셋인 HIV나 MUV 데이터셋을 Unlabeled Data로 사용해볼 수 있을 것 같습니다.